In [ ]:
from keras.callbacks import TensorBoard
from keras.models import load_model 
from keras import backend as K
import callbacks
import pandas as pd
import numpy as np
import scipy
import sys
import imp

In [ ]:
from process_data import load_data,get_rank_of_size
from data_util import unpack_news_data,data_generator
from layers import AttentionLayer
import models
import callbacks

imp.reload(models)
imp.reload(callbacks)

In [ ]:
rank = get_rank_of_size()
rank.remove('XOM')
rank.remove('GOOG')

In [ ]:
data = load_data(rank[:40],encoder_kind='Bert')

In [ ]:
data2 = load_data(rank[:20],encoder_kind='Glove')

In [ ]:
data3 = load_data(rank[:1],encoder_kind='Bert',with_date=True)

In [ ]:
K.clear_session()

In [ ]:
h_5 = load_model('model/h_5.h5', custom_objects = {'AttentionLayer': AttentionLayer})
h_10 = load_model('model/h_10.h5', custom_objects = {'AttentionLayer': AttentionLayer})
h_15 = load_model('model/h_15.h5', custom_objects = {'AttentionLayer': AttentionLayer})
t_5 = load_model('model/t_5.h5', custom_objects = {'AttentionLayer': AttentionLayer})
t_10 = load_model('model/t_10.h5', custom_objects = {'AttentionLayer': AttentionLayer})
t_15 = load_model('model/t_15.h5', custom_objects = {'AttentionLayer': AttentionLayer})
n_5 = load_model('model/n_5.h5', custom_objects = {'AttentionLayer': AttentionLayer})
n_10 = load_model('model/n_10.h5', custom_objects = {'AttentionLayer': AttentionLayer})
n_15 = load_model('model/n_15.h5', custom_objects = {'AttentionLayer': AttentionLayer})
n_20 = load_model('model/n_20.h5', custom_objects = {'AttentionLayer': AttentionLayer})
h_10_g = load_model('model/h_10_g.h5', custom_objects = {'AttentionLayer': AttentionLayer})
r_0 = models.build_guess_model()

In [ ]:
cbs = [callbacks.MetricsEx('mcc')]
def test(model,kind='H',dkind='test'):
    result_dict = {}
    for code in rank[21:31]:
        if kind == 'H':
            result = model.evaluate(x=[unpack_news_data(data[code][dkind][0]),data[code][dkind][1]],
                             y=data[code][dkind][2],batch_size=64,)
        if kind == 'N':
            result = model.evaluate(x=data[code][dkind][1],y=data[code][dkind][2],batch_size=64,)
        if kind == 'T':
            result = model.evaluate(x=unpack_news_data(data[code][dkind][0]),y=data[code][dkind][2],batch_size=64,)
        if kind == 'G':
            result = model.evaluate(x=np.random.randint(2, size=(data[code][dkind][2].shape[0],1)),
                                    y=data[code][dkind][2],batch_size=64)
        if kind == 'HG':
            result = model.evaluate(x=[unpack_news_data(data2[code][dkind][0]),data2[code][dkind][1]],
                             y=data2[code][dkind][2],batch_size=64)
        result_dict[code] = result[1]
        if 'avg' not in result_dict:
            result_dict['avg'] = 0
        result_dict['avg'] += result[1]
    result_dict['avg'] /= 10
    return result_dict

In [ ]:
test_dict = dict()
test_dict['h_5'] = test(h_5, kind='H')
test_dict['h_10'] = test(h_10, kind='H')
test_dict['h_15'] = test(h_15, kind='H')
test_dict['t_5'] = test(t_5, kind='T')
test_dict['t_10'] = test(t_10, kind='T')
test_dict['t_15'] = test(t_15, kind='T')
test_dict['N_5'] = test(n_5, kind='N')
test_dict['N_10'] = test(n_10, kind='N')
test_dict['N_15'] = test(n_15, kind='N')
test_dict['N_20'] = test(n_20, kind='N')


In [ ]:
np.random.seed(35)
test_dict['r_0'] = test(r_0,kind='G')
test_dict['h_10_g'] = test(h_10_g,kind='HG')
test2_dict['h_10'] =  test(h_10, kind='H',dkind='train')

In [ ]:
import csv
with open('result_g.csv','w',newline='') as f:
    header = list(test_dict['h_5'].keys())
    header.remove('avg')
    header.append('avg')
    writer = csv.DictWriter(f,header)
    writer.writeheader()
    writer.writerow(test_dict['h_10'])
    writer.writerow(test_dict['h_10_g'])

In [ ]:
import matplotlib.pyplot as plt
from numpy import arange
ppp = ['h_10', 't_10', 'N_20', 'r_0']

bar_widths = [test_dict[p]['avg'] for p in ppp]
bar_positions = arange(4) + 0.75
tick_positions = range(1,5)
fig, ax = plt.subplots()
ax.barh(bar_positions, bar_widths, 0.5)

ax.set_yticks(tick_positions)
ax.set_yticklabels(num_cols)
ax.set_ylabel('model objects')
ax.set_xlabel('average precision')

plt.show()